In [ ]:
%run ANNOTATIONS.ipynb

In [ ]:
%run PATTERN_OBSERVER.ipynb

In [ ]:
# ability to make backtest checkbox and book checkbox lists
%run CHECKBOXES.ipynb

In [ ]:
# location of datafiles
%run DATA_CONTAINER.ipynb

In [ ]:
# ability to make settings block
%run SETTINGS.ipynb

In [ ]:
# define extraday analysis plots
%run VIEWING_TABS_EXTRADAY_ANALYSIS.ipynb

In [ ]:
# define intraday analysis plots
%run VIEWING_TABS_INTRADAY_ANALYSIS.ipynb

In [ ]:
# define factor analysis plots
%run VIEWING_TABS_FACTOR_ANALYSIS.ipynb

In [ ]:
class BACKTEST_VIEWER(IObserver):
    def __init__(self,
        name='BACKTEST_VIEWER'
        ):
        self.name=name
        
        # instantiate SETTINGS
        self._settings=SETTINGS(
            name='SETTINGS',
            width='20%',
        )
        self._view_type=self.applied_view_type

        # instantiate BACKTEST_SELECTOR
        self._backtests=CHECKBOXES(
            name='BACKTEST_SELECTOR',
            options=self._settings.reference['all_backtests'],
            default=False,
            width='60%',
        )

        # instantiate BOOK_SELECTOR
        self._books=CHECKBOXES(
            name='BOOK_SELECTOR',
            options=self._settings.reference['all_books'],                                                           # TODO `BACKTEST SELECTOR` needs to have `all_books`
            default=True,
            width='20%',
        )
        #self._backtests.attach(self._books)                                                                         # TODO `BOOK SELECTOR` needs to observe changes in `BACKTEST SELECTOR|all_backtests`

        # instantiate DATA_CONTAINER
        self._data_container=DATA_CONTAINER(                                                                                                # TODO needs to observe changes in `self.selected_backtests`, `self.selected_books` and `self.applied_settings`
            applied_backtest_path=self._applied_backtest_path,
            applied_view_type=self._applied_view_type,
            applied_return_type=self._applied_return_type,
            applied_date_from=self._applied_date_from,
            applied_date_to=self._applied_date_to,
            selected_backtests=self._selected_backtests,
            selected_books=self._selected_books,
        )
        
        # BACKTEST_SELECTOR observes ...
        self._settings.attach(self._backtests)       # `all_backtests` from SETTINGS

        # DATA_CONTAINER observes ...
        self._settings.attach(self._data_container)  # `backtest_path` from SETTINGS
        self._backtests.attach(self._data_container) # `applied_settings` from BACKTEST_SELECTOR, i.e. changes in selected backtests
        self._books.attach(self._data_container)     # `applied_settings` from BOOK_SELECTOR, i.e. changes in selected books

        # build
        self.widget=self._make()                                                                                        # TODO needs to observe changes in self._data_container

    ####################################
    # observer pattern
    ####################################
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)

        # SETTINGS changed
        if subject_name=='SETTINGS':
            if self._view_type!=self.subject_info['view_type']:

                # VIEWING TABS observe DATA_CONTAINER
                self._data_container.attach(EXTRADAY_TABS)
                self._data_container.attach(INTRADAY_TABS)
                self._data_container.attach(FACTOR_TABS)

    @property
    def _applied_backtest_path(self) -> str :
        return self._settings.reference['backtest_path']

    @property
    def _applied_view_type(self) -> str :
        return self._settings.reference['view_type']

    @property
    def _applied_return_type(self) -> str :
        return self._settings.reference['return_type']

    @property
    def _applied_date_from(self) -> object :
        return self._settings.reference['date_from']

    @property
    def _applied_date_to(self) -> object :
        return self._settings.reference['date_to']

    @property
    def _selected_backtests(self) -> OPTIONID_VALUES:
        return self._backtests.applied_settings
        
    @property
    def _selected_books(self) -> OPTIONID_VALUES:
        return self._books.applied_settings

    @property
    def _control_widgets(self) -> BUTTONS:
        return self.widget.children[0].children
    
    @property
    def _viewing_widgets(self) -> WIDGETS:
        return self.widget.children[1].children

    ####################################
    # GUI
    ####################################
    def _make(self) -> VBox :
        # controls
        controls=HBox(
            [
                self._backtests.widget,
                self._books.widget,
                self._settings.widget,
            ]
        )
        
        # viewing gab
        self._make_viewing_tab()

        # build screen
        return VBox(
            [
                HBox(controls),
                self._viewing_block.widget
            ]
        )
    
    def _make_viewing_tab(self) -> TAB_COLLECTION :
        # instantiate
        self._viewing_tab=make_extraday_tab()
        
        # VIEWING_TAB observes ...
        self._settings.attach(self._viewing_tab)                  # `view_type` from SETTINGS

        # register checkboxes
        for c in widgets:
            c.observe(self._onchange_checkbox,names='value')

# example
BACKTEST_VIEWER().widget

In [ ]:
# display
a=BACKTEST_VIEWER()
a.widget

In [ ]:
import pickle
a_file = open("data.pkl", "wb")
pickle.dump(dat,a_file)
'''
import pickle
a_file = open("data.pkl", "rb")
output = pickle. load(a_file)
'''